In [10]:
import glob, os
import shutil
import matplotlib.pyplot as plt
import pprint
import numpy as np

from pathlib import Path

import pandas as pd

from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
'''
'''
#TODO: export requirements

Using TensorFlow backend.


'\n'

In [11]:
#Load Dataset
'''
According to esc50 documentation rooster class is = 1.
'''
#50 CLASSES SOUND DATA SET
AUDIO_DS_PATH = Path("Data/Dataset/ESC-50")
DF_PATH =  Path("Data/Dataset/ESC-50-master/meta/esc50.csv")

#Dataset as pandas dataframe
sdf = pd.read_csv(DF_PATH)

print("Number of entries in Data frame: ", len(sdf.index))
# Count number of rows in a dataframe that contains NaN any column
seriesObj = sdf.apply(lambda x: x.isnull().any(), axis=1)
numOfRows = len(seriesObj[seriesObj == True].index)
print('Number of Rows in dataframe which contain NaN in any column : ', numOfRows)

#PRINT COUNT Fold SETS
seriesObj = sdf.apply(lambda x: True if x['fold'] == 1 else False , axis=1)
# Count number of True in series
numOfRows = len(seriesObj[seriesObj == True].index)
#target classes non categorical indexes easier to work with keras
classes = sorted(sdf.target.unique())

print('Number of folds: ', len(sdf.fold.unique()))
print('Count per fold: ', numOfRows)
print('Classes indexes in non categorical index: ', classes )

#clean data set
# Removed some columsn that don not seem important
sdf = sdf.drop(['take','src_file', 'category','esc10'], axis=1)
sdf.head()

Number of entries in Data frame:  2000
Number of Rows in dataframe which contain NaN in any column :  0
Number of folds:  5
Count per fold:  400
Classes indexes in non categorical index:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


,filename,fold,target
0,1-100032-A-0.wav,1,0
1,1-100038-A-14.wav,1,14
2,1-100210-A-36.wav,1,36
3,1-100210-B-36.wav,1,36
4,1-101296-A-19.wav,1,19


In [12]:
#kfold-spliting take advantage of dataste fold column
kf = KFold(n_splits = 5,shuffle=True, random_state=1)
def get_fold_from(dataframe):
    result = next(kf.split(dataframe), None)
    train = df.iloc(result[0])
    test =

TypeError: __init__() got an unexpected keyword argument 'shiffle'